In [1]:
import pandas as pd
import sklearn as sk
from matplotlib.pylab import plt
import numpy as np
%matplotlib inline
from sklearn.externals import joblib

In [2]:
df_X = pd.read_pickle('df_X.p')

In [3]:
y = np.load('y.npy')
X = np.load('X.npy')

In [4]:
def last_time_split(ratio, *dfs):
    pct = int(dfs[0].shape[0] * ratio)
    trains = [d[:-pct] for d in dfs]
    tests = [d[-pct:] for d in dfs]
    return trains + tests

In [5]:
X_train, y_train, X_test, y_test = last_time_split(0.1, X, y)

In [6]:
from xgboost import XGBRegressor

c:\Users\weixi51\AppData\Local\Continuum\Anaconda\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
xgb = XGBRegressor(seed=0, silent=False)

Since there are way too many parameters to tune at once. I will grid search twice for hyper parameters using guide at
https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/

In [9]:
from sklearn.model_selection import TimeSeriesSplit
tss = TimeSeriesSplit(n_splits=5)

In [10]:
from sklearn.model_selection import GridSearchCV

In [8]:
param_test1 = {
 'max_depth':range(3,16,2),
 'min_child_weight':range(1,10,2)
}

In [12]:
gsearch1 = GridSearchCV(XGBRegressor(seed=0),
                        param_grid=param_test1, scoring='r2', cv=tss, iid=True, n_jobs=4)

In [13]:
gsearch1.fit(X_train, y_train)

GridSearchCV(cv=TimeSeriesSplit(n_splits=5), error_score='raise',
       estimator=XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1),
       fit_params={}, iid=True, n_jobs=4,
       param_grid={'max_depth': [3, 5, 7, 9, 11, 13, 15], 'min_child_weight': [1, 3, 5, 7, 9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='r2', verbose=0)

In [16]:
gsearch1.best_score_

-0.0054601171793367429

It looks like noe of the models are performing better than base prediction of 0. Let's try again

In [17]:
gsearch2 = GridSearchCV(XGBRegressor(seed=0, max_depth=3),
                        param_grid={'min_child_weight': range(9, 20, 2)}, scoring='r2', cv=tss, n_jobs=4)

In [18]:
gsearch2.fit(X_train, y_train)

GridSearchCV(cv=TimeSeriesSplit(n_splits=5), error_score='raise',
       estimator=XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1),
       fit_params={}, iid=True, n_jobs=4,
       param_grid={'min_child_weight': [9, 11, 13, 15, 17, 19]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='r2', verbose=0)

In [19]:
print gsearch2.best_estimator_,'\n', gsearch2.best_score_

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=13, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1) 
-0.00374649458228


In [20]:
param_3 = {'max_depth': [3, 4], 'min_child_weight': [12, 13, 14]}
gsearch3 = GridSearchCV(XGBRegressor(seed=0, max_depth=3),
                        param_grid=param_3, scoring='r2', cv=tss, n_jobs=4)

In [21]:
gsearch3.fit(X_train, y_train)

GridSearchCV(cv=TimeSeriesSplit(n_splits=5), error_score='raise',
       estimator=XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1),
       fit_params={}, iid=True, n_jobs=4,
       param_grid={'max_depth': [3, 4], 'min_child_weight': [12, 13, 14]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='r2', verbose=0)

In [22]:
print gsearch3.best_estimator_,'\n', gsearch3.best_score_

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=13, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1) 
-0.00374649458228


use max_depth=3 and min_child_weight=13 for subsequent CV

In [24]:
param_4 = {'gamma': [0.0, 0.2, 0.4, 0.6, 0.8], 'subsample': [0.6, 0.8, 1.0]}
gsearch4 = GridSearchCV(XGBRegressor(seed=0, max_depth=3, min_child_weight=13),
                        param_grid=param_4, scoring='r2', cv=tss, n_jobs=4)

In [26]:
gsearch4.fit(X_train, y_train)

GridSearchCV(cv=TimeSeriesSplit(n_splits=5), error_score='raise',
       estimator=XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=13, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1),
       fit_params={}, iid=True, n_jobs=4,
       param_grid={'subsample': [0.6, 0.8, 1.0], 'gamma': [0.0, 0.2, 0.4, 0.6, 0.8]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='r2', verbose=0)

In [27]:
print gsearch4.best_estimator_,'\n', gsearch4.best_score_

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0.4, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=13, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1.0) 
0.00419593854898


In [31]:
param_5 = {'gamma': [0.3, 0.4, 0.5], 'subsample': [0.9, 1.0]}
gsearch5 = GridSearchCV(XGBRegressor(seed=0, max_depth=3, min_child_weight=13),
                        param_grid=param_5, scoring='r2', cv=tss, n_jobs=4)

In [32]:
gsearch5.fit(X_train, y_train)

GridSearchCV(cv=TimeSeriesSplit(n_splits=5), error_score='raise',
       estimator=XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=13, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1),
       fit_params={}, iid=True, n_jobs=4,
       param_grid={'subsample': [0.9, 1.0], 'gamma': [0.3, 0.4, 0.5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='r2', verbose=0)

In [33]:
print gsearch5.best_estimator_,'\n', gsearch5.best_score_

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0.4, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=13, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1.0) 
0.00419593854898


In [34]:
param_6 = {'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0]}
gsearch6 = GridSearchCV(XGBRegressor(seed=0, max_depth=3, min_child_weight=13, gamma=0.4, subsample=1.0),
                        param_grid=param_6, scoring='r2', cv=tss, n_jobs=4)

In [35]:
gsearch6.fit(X_train, y_train)

GridSearchCV(cv=TimeSeriesSplit(n_splits=5), error_score='raise',
       estimator=XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0.4, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=13, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1.0),
       fit_params={}, iid=True, n_jobs=4,
       param_grid={'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='r2', verbose=0)

In [37]:
print gsearch6.best_params_,'\n', gsearch6.best_score_

{'colsample_bytree': 1.0} 
0.00419593854898


In [39]:
param_7 = {
 'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
}
gsearch7 = GridSearchCV(XGBRegressor(
    seed=0, max_depth=3, min_child_weight=13, gamma=0.4, subsample=1.0, colsample_bytree=1),
                        param_grid=param_7, scoring='r2', cv=tss, n_jobs=4)

In [40]:
gsearch7.fit(X_train, y_train)

GridSearchCV(cv=TimeSeriesSplit(n_splits=5), error_score='raise',
       estimator=XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0.4, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=13, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1.0),
       fit_params={}, iid=True, n_jobs=4,
       param_grid={'reg_alpha': [1e-05, 0.01, 0.1, 1, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='r2', verbose=0)

In [41]:
print gsearch7.best_params_,'\n', gsearch7.best_score_

{'reg_alpha': 1} 
0.00484998937221


In [42]:
param_8 = {
 'reg_alpha':[0.5, 1, 5, 10, 20]
}
gsearch8 = GridSearchCV(XGBRegressor(
    seed=0, max_depth=3, min_child_weight=13, gamma=0.4, subsample=1.0, colsample_bytree=1),
                        param_grid=param_8, scoring='r2', cv=tss, n_jobs=4)

In [43]:
gsearch8.fit(X_train, y_train)

GridSearchCV(cv=TimeSeriesSplit(n_splits=5), error_score='raise',
       estimator=XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0.4, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=13, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1.0),
       fit_params={}, iid=True, n_jobs=4,
       param_grid={'reg_alpha': [0.5, 1, 5, 10, 20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='r2', verbose=0)

In [44]:
print gsearch8.best_params_,'\n', gsearch8.best_score_

{'reg_alpha': 5} 
0.00546684262418


one final gridsearch

In [45]:
param_9 = {
    'reg_alpha': [4, 5, 6],
    'gamma': [0.35, 0.4, 0.45],
    'max_depth': [3, 4],
    'min_child_weight': [12, 13, 14]
}

gsearch9 = GridSearchCV(XGBRegressor(
    seed=0, subsample=1.0, colsample_bytree=1),
                        param_grid=param_9, scoring='r2', cv=tss, n_jobs=4)

In [ ]:
from sklearn.metrics import mean_absolute_error